In [1]:
import threading
import time
import cv2
import tensorflow as tf
import numpy as np
import os
from queue import Queue
from retinaface import RetinaFace

# Enable GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("[INFO] TensorFlow is using GPU:", tf.config.list_physical_devices('GPU'))
    except RuntimeError as e:
        print(e)

# Camera sources
CAMERA_SOURCES = {
    "camera1": "http://10.24.75.85:4747/video",
}
frames = {name: Queue(maxsize=1) for name in CAMERA_SOURCES}
faces_queue = Queue(maxsize=10)  # Queue to send detected faces to embeddings script

# Directory to save detected faces
SAVE_PATH = "detected_faces"
os.makedirs(SAVE_PATH, exist_ok=True)

# Stores last saved time for each camera (to prevent duplicate saves)
last_saved_time = {}
last_frame_time = {}  # Tracks the last valid frame timestamp


def detect_faces(frame, camera_name):
    """Detect faces, save unique images, and update GUI."""
    global last_saved_time

    if frame is None or frame.size == 0:
        return frame  # Continue running even if no face is detected

    faces = RetinaFace.detect_faces(frame)

    if isinstance(faces, dict) and faces:
        for face_id, face in faces.items():
            x1, y1, x2, y2 = face["facial_area"]
            cropped_face = frame[y1:y2, x1:x2]

            # Save only if 5 seconds have passed since last save
            current_time = time.time()
            if camera_name not in last_saved_time or (current_time - last_saved_time[camera_name] > 5):
                face_filename = os.path.join(SAVE_PATH, f"face_{int(current_time)}.jpg")
                cv2.imwrite(face_filename, cropped_face)
                last_saved_time[camera_name] = current_time  # Update last saved time
                print(f"[INFO] Saved: {face_filename}")

                try:
                    faces_queue.put_nowait((cropped_face, face_filename))  # Send to embedding script
                except:
                    pass  

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    return frame


def capture_and_process_camera(camera_name, url):
    """Capture video and process frames."""
    global last_frame_time

    cap = cv2.VideoCapture(url)
    if not cap.isOpened():
        print(f"[ERROR] Unable to open {camera_name}")
        return

    last_frame_time[camera_name] = time.time()  # Initialize last frame time

    while True:
        ret, frame = cap.read()

        if not ret:
            print(f"[WARNING] No frame captured from {camera_name}")
            if time.time() - last_frame_time[camera_name] > 3:  # Check if 3 seconds passed
                print(f"[ERROR] No frames received for 3 seconds from {camera_name}. Exiting...")
                break  # Exit loop if timeout is exceeded
            continue  # Keep trying

        last_frame_time[camera_name] = time.time()  # Update last frame timestamp
        frame = cv2.resize(frame, (640, 480))
        processed_frame = detect_faces(frame, camera_name)

        try:
            frames[camera_name].put_nowait(processed_frame)
        except:
            pass  

    cap.release()
    print(f"[INFO] Camera {camera_name} thread stopped.")


def cleanup_old_faces():
    """Periodically delete processed face images after embedding extraction."""
    while True:
        time.sleep(10)  # Run cleanup every 10 seconds
        for face_image in os.listdir(SAVE_PATH):
            image_path = os.path.join(SAVE_PATH, face_image)
            if os.path.exists(image_path):
                os.remove(image_path)
                print(f"[CLEANUP] Deleted: {image_path}")


# Start camera thread
threads = []
for name, url in CAMERA_SOURCES.items():
    thread = threading.Thread(target=capture_and_process_camera, args=(name, url), daemon=True)
    threads.append(thread)
    thread.start()
    time.sleep(1)

# Start cleanup thread
cleanup_thread = threading.Thread(target=cleanup_old_faces, daemon=True)
cleanup_thread.start()

# Display video
while True:
    active_threads = 0
    for name in CAMERA_SOURCES.keys():
        try:
            frame = frames[name].get_nowait()
            if frame is not None:
                cv2.imshow(f"Live Stream - {name}", frame)
                active_threads += 1
        except:
            pass  

    # If no active threads remain, exit
    if active_threads == 0:
        print("[INFO] All camera threads stopped. Exiting program...")
        break

    if cv2.waitKey(1) & 0xFF == ord("q"):
        print("[INFO] Manual exit triggered.")
        break

cv2.destroyAllWindows()


2025-03-08 15:14:22.311685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-08 15:14:22.449925: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 15:14:22.485666: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 15:14:23.172373: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

[INFO] TensorFlow is using GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[INFO] All camera threads stopped. Exiting program...


2025-03-08 15:14:27.039160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-08 15:14:27.046032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-08 15:14:27.046135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-03-08 15:14:27.046175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may h

[INFO] Saved: detected_faces/face_1741427079.jpg
[INFO] Saved: detected_faces/face_1741427084.jpg
[CLEANUP] Deleted: detected_faces/face_1741427079.jpg
[CLEANUP] Deleted: detected_faces/face_1741427084.jpg
[INFO] Saved: detected_faces/face_1741427089.jpg
[INFO] Saved: detected_faces/face_1741427094.jpg
[CLEANUP] Deleted: detected_faces/face_1741427089.jpg
[CLEANUP] Deleted: detected_faces/face_1741427094.jpg
[INFO] Saved: detected_faces/face_1741427099.jpg
[INFO] Saved: detected_faces/face_1741427104.jpg
[CLEANUP] Deleted: detected_faces/face_1741427099.jpg
[CLEANUP] Deleted: detected_faces/face_1741427104.jpg
[INFO] Saved: detected_faces/face_1741427109.jpg
[INFO] Saved: detected_faces/face_1741427114.jpg
[CLEANUP] Deleted: detected_faces/face_1741427109.jpg
[CLEANUP] Deleted: detected_faces/face_1741427114.jpg
[INFO] Saved: detected_faces/face_1741427119.jpg
[INFO] Saved: detected_faces/face_1741427124.jpg
[CLEANUP] Deleted: detected_faces/face_1741427119.jpg
[CLEANUP] Deleted: detec

[http @ 0x7f0c10001cc0] Stream ends prematurely at 9644596, should be 18446744073709551615
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No frame captured from camera1
[WARNING] No fra